In [2]:
!pip install boto3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.7/134.7 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.5/10.5 MB 55.8 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.6/79.6 kB 3.7 MB/s eta 0:00:00

[notice] A new release of pip is available: 23.0 -> 23.0.1
[notice] To update, run: python -m pip install --upgrade pip


In [3]:
import boto3
import pandas as pd

# Set the parameters
database = 'test_table'
table = 'your-athena-table'
s3_staging_dir = 's3://your-s3-bucket/athena-staging/'
region = 'your-aws-region'

# Create a client for Athena
athena = boto3.client('athena', region_name=region)

# Submit a query to Athena
query = f"SELECT * FROM {database}.{table}"
response = athena.start_query_execution(
    QueryString=query,
    QueryExecutionContext={
        'Database': database
    },
    ResultConfiguration={
        'OutputLocation': s3_staging_dir
    }
)

# Wait for the query to complete
query_execution_id = response['QueryExecutionId']
status = 'RUNNING'
while status in ['RUNNING', 'QUEUED']:
    response = athena.get_query_execution(QueryExecutionId=query_execution_id)
    status = response['QueryExecution']['Status']['State']

# Retrieve the query results
response = athena.get_query_results(QueryExecutionId=query_execution_id)
rows = response['ResultSet']['Rows']

# Create a Pandas DataFrame from the rows
data = [list(row['Data'].values()) for row in rows]
df = pd.DataFrame(data[1:], columns=data[0])

# Print the DataFrame
print(df.head())

In [ ]:
import pandas as pd
from get_s3_data import get_s3_data

# Set the parameters
bucket = 'your-s3-bucket'
key = 'path/to/your/data.csv'
s3_staging_dir = 's3://your-s3-bucket/athena-staging/'
region = 'your-aws-region'
database = 'your-athena-database'
table = 'your-athena-table'

# Call the function
df = get_s3_data(bucket, key, s3_staging_dir, region, database, table)

# Print the DataFrame
print(df.head())


